<a href="https://colab.research.google.com/github/JimmyOlsen05/Autism/blob/main/OVERALL_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Concatenate, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import image
!pip install tensorflow joblib tf2onnx

import numpy as np
!pip install dill
import dill

BUILD AND TRAIN LSTM MODEL


In [ ]:


# Load the M-CHAT dataset
data = pd.read_csv('/content/drive/MyDrive/Toddler Autism dataset July 2018.csv')

# Drop the 'Qchat-10-Score' column as recommended
data = data.drop('Qchat-10-Score', axis=1)

# Encode the target variable ('Class/ASD Traits')
label_encoder = LabelEncoder()
data['Class/ASD Traits '] = label_encoder.fit_transform(data['Class/ASD Traits '])

# Encode the categorical features
categorical_cols = ['Sex', 'Ethnicity', 'Jaundice', 'Family_mem_with_ASD', 'Who_completed_the_test']
data = pd.get_dummies(data, columns=categorical_cols)

# Convert boolean columns to integers
bool_columns = ['Sex_f', 'Sex_m', 'Ethnicity_Hispanic', 'Ethnicity_Latino', 'Ethnicity_Native Indian', 'Ethnicity_Others', 'Ethnicity_Pacifica', 'Ethnicity_White European', 'Ethnicity_asian', 'Ethnicity_black', 'Ethnicity_middle eastern', 'Ethnicity_mixed', 'Ethnicity_south asian', 'Jaundice_no', 'Jaundice_yes', 'Family_mem_with_ASD_no', 'Family_mem_with_ASD_yes', 'Who_completed_the_test_Health Care Professional', 'Who_completed_the_test_Health care professional', 'Who_completed_the_test_Others', 'Who_completed_the_test_Self', 'Who_completed_the_test_family member']
data[bool_columns] = data[bool_columns].astype(int)

# Extract the sequence data and categorical features
sequences = data[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10']].values.tolist()
categorical_features = data.drop(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Class/ASD Traits '], axis=1)

# Encode the sequences
encoded_sequences = []
for sequence in sequences:
    encoded_seq = [int(value) for value in sequence]
    encoded_sequences.append(encoded_seq)

# Pad/truncate sequences to a fixed length
max_sequence_length = max(len(seq) for seq in encoded_sequences)
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Split the data into features (X) and target (y)
X = [padded_sequences, categorical_features]
y = data['Class/ASD Traits ']



# Split the data into train, validation, and test sets
train_indices, test_indices = train_test_split(data.index, test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_indices, test_size=0.2, random_state=42)

X_train = [X[0][train_idx] for train_idx in train_indices]
X_val = [X[0][val_idx] for val_idx in val_indices]
X_test = [X[0][test_idx] for test_idx in test_indices]

X_train_cat = X[1].iloc[train_indices]
X_val_cat = X[1].iloc[val_indices]
X_test_cat = X[1].iloc[test_indices]

# Split the target variable into train, validation, and test sets
y_train = y[train_indices]
y_val = y[val_indices]
y_test = y[test_indices]



# Convert boolean columns to integers
X_train_cat_int = X_train_cat.astype(int)
X_val_cat_int = X_val_cat.astype(int)
X_test_cat_int = X_test_cat.astype(int)

# Reshape sequence data to include batch size and timestep dimensions
X_train_seq_reshaped = np.expand_dims(X_train, axis=-1)
X_val_seq_reshaped = np.expand_dims(X_val, axis=-1)
X_test_seq_reshaped = np.expand_dims(X_test, axis=-1)

# Define input layers for sequence data and categorical features
sequence_input = Input(shape=(max_sequence_length, 1), name='sequence_input')
cat_input = Input(shape=(X_train_cat_int.shape[1],), name='cat_input')

# LSTM model for sequence data
lstm_layer = LSTM(64)(sequence_input)

# Concatenate LSTM output with categorical input
concatenated = Concatenate()([lstm_layer, cat_input])

# Dense layers
dense1 = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='sigmoid')(dense1)

# Define the model
lstm_model = Model(inputs=[sequence_input, cat_input], outputs=output)

# Compile the model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
lstm_model.summary()

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_.h5', monitor='val_loss', save_best_only=True, verbose=1)

history = lstm_model.fit([X_train_seq_reshaped, X_train_cat_int], y_train, epochs=10, batch_size=32,
                    validation_data=([X_val_seq_reshaped, X_val_cat_int], y_val),
                    callbacks=[early_stop, checkpoint])

# Evaluate the model
loss, accuracy = lstm_model.evaluate([X_test_seq_reshaped, X_test_cat_int], y_test)
print(f'Test accuracy: {accuracy}')



# Save the model
lstm_model.save('/content/drive/MyDrive/LSTM_model.h5')
with open('/content/drive/MyDrive/autism_data.pkl', 'wb') as f:
    dill.dump((encoded_sequences, categorical_cols, max_sequence_length,X_train_cat), f)

In [ ]:
# Save the model
lstm_model.save('/content/drive/MyDrive/LSTM_model.h5')

import tensorflow as tf
import pickle
from joblib import dump
import tf2onnx
import onnx
import os

# Assuming 'lstm_model' is your trained Keras LSTM model
# lstm_model = ...

# Define the paths for saving the model in different formats
base_path = '/content/drive/MyDrive'
h5_path = os.path.join(base_path, 'LSTM_model.h5')
pickle_path = os.path.join(base_path, 'LSTM_model.pkl')
joblib_path = os.path.join(base_path, 'LSTM_model.joblib')
onnx_path = os.path.join(base_path, 'LSTM_model.onnx')
saved_model_path = os.path.join(base_path, 'saved_model')
json_path = os.path.join(base_path, 'LSTM_model.json')
weights_path = os.path.join(base_path, 'LSTM_model_weights.h5')

# Save as HDF5 (.h5)
lstm_model.save(h5_path)
print(f"Model saved as HDF5 at {h5_path}")

# Save as Pickle (.pkl)
with open(pickle_path, 'wb') as file:
    pickle.dump(lstm_model, file)
print(f"Model saved as Pickle at {pickle_path}")

# Save as Joblib (.joblib)
dump(lstm_model, joblib_path)
print(f"Model saved as Joblib at {joblib_path}")

# Save as ONNX (.onnx)
onnx_model, _ = tf2onnx.convert.from_keras(lstm_model)
onnx.save(onnx_model, onnx_path)
print(f"Model saved as ONNX at {onnx_path}")

# Save as TensorFlow SavedModel
lstm_model.save(saved_model_path)
print(f"Model saved as TensorFlow SavedModel at {saved_model_path}")

# Save model architecture as JSON and weights as HDF5
model_json = lstm_model.to_json()
with open(json_path, "w") as json_file:
    json_file.write(model_json)
lstm_model.save_weights(weights_path)
print(f"Model architecture saved as JSON at {json_path}")
print(f"Model weights saved as HDF5 at {weights_path}")

# Note: PMML saving process is omitted here due to potential compatibility issues.
# If your model is compatible, you can use the sklearn2pmml or other appropriate tools for conversion.

BUILD AND TRAIN CNN MODEL

In [ ]:
# Define data paths (adjust according to your directory structure)
train_data_dir = '/content/drive/MyDrive/NEW_DATASET_DIRECTORY/data/train'
test_data_dir = '/content/drive/MyDrive/NEW_DATASET_DIRECTORY/data/test'
validation_data_dir = '/content/drive/MyDrive/NEW_DATASET_DIRECTORY/data/valid'

# Set image dimensions (modify if needed)
img_width, img_height = 150, 150

# Data preprocessing and augmentation (consider these techniques)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # Adjust batch size as needed
    class_mode='binary'  # Binary classification for autistic/non-autistic
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # Adjust batch size as needed
    class_mode='binary'
)

# Define CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Model compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training (adjust epochs and other hyperparameters as needed)
history = model.fit(train_generator,
                    epochs=20,  # Adjust based on training time and validation performance
                    validation_data=validation_generator)

In [ ]:
model.save("/content/drive/MyDrive/CNN_MODEL_.h5")

import tensorflow as tf
import pickle
from joblib import dump
import tf2onnx
import onnx
import os

# Assuming 'model' is your trained Keras model
# model = ...

# Define the paths for saving the model in different formats
base_path = '/content/drive/MyDrive'
h5_path = os.path.join(base_path, 'CNN_MODEL_.h5')
pickle_path = os.path.join(base_path, 'CNN_MODEL_.pkl')
joblib_path = os.path.join(base_path, 'CNN_MODEL_.joblib')
onnx_path = os.path.join(base_path, 'CNN_MODEL_.onnx')
saved_model_path = os.path.join(base_path, 'saved_model')
json_path = os.path.join(base_path, 'CNN_MODEL_.json')
weights_path = os.path.join(base_path, 'CNN_MODEL_weights_.h5')

# Save as HDF5 (.h5)
model.save(h5_path)
print(f"Model saved as HDF5 at {h5_path}")

# Save as Pickle (.pkl)
with open(pickle_path, 'wb') as file:
    pickle.dump(model, file)
print(f"Model saved as Pickle at {pickle_path}")

# Save as Joblib (.joblib)
dump(model, joblib_path)
print(f"Model saved as Joblib at {joblib_path}")

# Save as ONNX (.onnx)
onnx_model, _ = tf2onnx.convert.from_keras(model)
onnx.save(onnx_model, onnx_path)
print(f"Model saved as ONNX at {onnx_path}")

# Save as TensorFlow SavedModel
model.save(saved_model_path)
print(f"Model saved as TensorFlow SavedModel at {saved_model_path}")

# Save model architecture as JSON and weights as HDF5
model_json = model.to_json()
with open(json_path, "w") as json_file:
    json_file.write(model_json)
model.save_weights(weights_path)
print(f"Model architecture saved as JSON at {json_path}")
print(f"Model weights saved as HDF5 at {weights_path}")

# Note: PMML saving process is omitted here due to potential compatibility issues.
# If your model is compatible, you can use the sklearn2pmml or other appropriate tools for conversion.

In [ ]:
lstm_model=load_model("/content/drive/MyDrive/LSTM_model.h5")
#cnn_model=load_model("/content/drive/MyDrive/CNN_MODEL_.h5")
cnn_model=load_model("/content/drive/MyDrive/CNN_MODEL_.json")


In [ ]:
'''import dill
# Load the trained LSTM model
lstm_model = load_model('/content/drive/MyDrive/LSTM_model.h5')


# Load the trained CNN model
cnn_model = load_model('/content/drive/MyDrive/CNN_MODEL_.json')
import dill
with open('/content/drive/MyDrive/autism_data.pkl', 'rb') as f:
    encoded_sequences, categorical_cols, max_sequence_length,X_train_cat = dill.load(f)

def predict_with_lstm(sequence, categorical_features):
    padded_sequence = pad_sequences([sequence], maxlen=max_sequence_length, padding='post', truncating='post')
    padded_sequence_reshaped = np.expand_dims(padded_sequence, axis=-1)
    categorical_features_int = np.array([categorical_features]).astype(int)

    prediction = lstm_model.predict([padded_sequence_reshaped, categorical_features_int])
    return prediction[0][0]

img_width=150
img_height=150

def predict_with_cnn(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x /= 255.0  # Rescale if necessary
    x = np.expand_dims(x, axis=0)  # Add batch dimension
    prediction = cnn_model.predict(x)
    return prediction[0][0]

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Create input layers for the meta-model
lstm_input = Input(shape=(1,), name='lstm_input')
cnn_input = Input(shape=(1,), name='cnn_input')

# Concatenate the inputs
concatenated = Concatenate()([lstm_input, cnn_input])

# Add a dense layer and output layer
dense = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='sigmoid')(dense)

# Create the meta-model
meta_model = Model(inputs=[lstm_input, cnn_input], outputs=output)

# Compile the meta-model
meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])





# Prepare the input for the LSTM model
sequence = [1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
categorical_features = [0] * len(X_train_cat.columns)

# Get the prediction from the LSTM model
lstm_prediction = predict_with_lstm(sequence, categorical_features)

# Prepare the input for the CNN model
image_path = '/content/drive/MyDrive/DATASET/Non-Autistic-2.jpg'

# Get the prediction from the CNN model
cnn_prediction = predict_with_cnn(image_path)

# Make a prediction using the ensemble model
ensemble_prediction = meta_model.predict([np.array([lstm_prediction]), np.array([cnn_prediction])])

# Convert the prediction to class label (0 or 1)
predicted_class = 1 if ensemble_prediction > 0.5 else 0
if predicted_class==1:
  print("Autistic")
else:
  print("Non-Autistic")
print("Predicted Class (Ensemble):", predicted_class)
meta_model.save('/content/drive/MyDrive/Meta_Model.save.h5')'''

In [ ]:
import numpy as np
import dill
from keras.models import load_model, model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import image
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Load the trained LSTM model
lstm_model = load_model('/content/drive/MyDrive/LSTM_model.h5')

# Load the trained CNN model from JSON
with open('/content/drive/MyDrive/CNN_MODEL_.json', 'r') as json_file:
    cnn_model_json = json_file.read()
cnn_model = model_from_json(cnn_model_json)
cnn_model.load_weights('/content/drive/MyDrive/CNN_MODEL_weights_.h5')

# Load the data
with open('/content/drive/MyDrive/autism_data.pkl', 'rb') as f:
    encoded_sequences, categorical_cols, max_sequence_length, X_train_cat = dill.load(f)

# Define prediction functions
def predict_with_lstm(sequence, categorical_features, lstm_model, max_sequence_length):
    padded_sequence = pad_sequences([sequence], maxlen=max_sequence_length, padding='post', truncating='post')
    padded_sequence_reshaped = np.expand_dims(padded_sequence, axis=-1)
    categorical_features_int = np.array([categorical_features]).astype(int)
    prediction = lstm_model.predict([padded_sequence_reshaped, categorical_features_int])
    return prediction[0][0]

img_width = 150
img_height = 150

def predict_with_cnn(image_path, cnn_model):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x /= 255.0  # Rescale if necessary
    x = np.expand_dims(x, axis=0)  # Add batch dimension
    prediction = cnn_model.predict(x)
    return prediction[0][0]

# Create and compile the meta-model
lstm_input = Input(shape=(1,), name='lstm_input')
cnn_input = Input(shape=(1,), name='cnn_input')
concatenated = Concatenate()([lstm_input, cnn_input])
dense = Dense(64, activation='relu')(concatenated)
output = Dense(1, activation='sigmoid')(dense)
meta_model = Model(inputs=[lstm_input, cnn_input], outputs=output)
meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare the input for the LSTM model
sequence = [1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
categorical_features = [0] * len(X_train_cat.columns)

# Get the prediction from the LSTM model
lstm_prediction = predict_with_lstm(sequence, categorical_features, lstm_model, max_sequence_length)

# Prepare the input for the CNN model
image_path = '/content/drive/MyDrive/NEW_DATASET_DIRECTORY/data/train/Autistic/980.jpg'

# Get the prediction from the CNN model
cnn_prediction = predict_with_cnn(image_path, cnn_model)

# Make a prediction using the ensemble model
ensemble_prediction = meta_model.predict([np.array([lstm_prediction]), np.array([cnn_prediction])])

# Convert the prediction to class label (0 or 1)
predicted_class = 1 if ensemble_prediction > 0.5 else 0
if predicted_class == 1:
    print("Autistic")
else:
    print("Non-Autistic")

print("Predicted Class (Ensemble):", predicted_class)

# Save the meta-model
# meta_model.save('/content/drive/MyDrive/Meta_Model_.h5')


1/1 [==============================] - 0s 57ms/step
Non-Autistic
Predicted Class (Ensemble): 0
